#Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importing the packages

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import keras
import sklearn
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,plot_roc_curve,log_loss,f1_score,precision_score,recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
import seaborn as sn
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Flatten, Dropout,LeakyReLU
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report,roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from keras.models import Sequential
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import roc_auc_score
warnings.simplefilter('ignore')

%matplotlib inline


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


### Importing the dataset

In [3]:
df = pd.read_csv('/content/drive/MyDrive/COVID19/Covid-19 Dataset_Clinical Data/ISRmergedtrain+test.csv')

#Split dataset 

In [4]:
X=df.values[:,0:8]
y=df.values[:,8]

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30)

#SMOTE

In [6]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1))) 
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0))) 
  

Before OverSampling, counts of label '1': 5952
Before OverSampling, counts of label '0': 63510 



In [7]:
# import SMOTE module from imblearn library 
# pip install imblearn (if you don't have imblearn in your system) 
from imblearn.over_sampling import SMOTE 
sm = SMOTE() 
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel()) 
  
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape)) 
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape)) 
  
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1))) 
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0))) 

After OverSampling, the shape of train_X: (127020, 8)
After OverSampling, the shape of train_y: (127020,) 

After OverSampling, counts of label '1': 63510
After OverSampling, counts of label '0': 63510


In [8]:
feat_labels=['AGE', 'GENDER', 'FEVER', 'COUGH', 'FATIGUE', 'PAINS',
       'NASAL_CONGESTION', 'SHORTNESS_OF_BREATH', 'RUNNY_NOSE', 'SORE THROAT',
       'DIARRHEA', 'CHILLS', 'HEADACHE', 'VOMITING', 'LIVES_IN_AFFECTED_AREA']

#Metrics

In [9]:
def calculate_sensitivity_specificity(y_test, y_pred_test):
    # Note: More parameters are defined than necessary. 
    # This would allow return of other measures other than sensitivity and specificity
    
    # Get true/false for whether a breach actually occurred
    actual_pos = y_test == 1
    actual_neg = y_test == 0
    
    # Get true and false test (true test match actual, false tests differ from actual)
    true_pos = (y_pred_test == 1) & (actual_pos)
    false_pos = (y_pred_test == 1) & (actual_neg)
    true_neg = (y_pred_test == 0) & (actual_neg)
    false_neg = (y_pred_test == 0) & (actual_pos)
    
    # Calculate accuracy
    accuracy = np.mean(y_pred_test == y_test)
    
    # Calculate sensitivity and specificity
    sensitivity = np.sum(true_pos) / np.sum(actual_pos)
    specificity = np.sum(true_neg) / np.sum(actual_neg)
    
    return sensitivity, specificity, accuracy

#SFFS

In [10]:
# for sfs
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
#knn = KNeighborsClassifier(n_neighbors=3) # ml_algo used = knn
#clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
clf = AdaBoostClassifier(n_estimators=100,learning_rate=0.1)
sfs1 = SFS(clf,  
           k_features=5, 
           forward=True, # if forward = True then SFS otherwise SBS
           floating=True, 
           verbose=2,
           scoring='accuracy', cv=0)
#after applying sfs fit the data:
sfs1.fit(X_train, y_train)
sfs1.k_feature_names_  


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   19.4s finished

[2021-02-21 17:17:47] Features: 1/5 -- score: 0.9406725979672339[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:   17.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s finished

[2021-02-21 17:18:06] Features: 2/5 -- score: 0.9492240361636578[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s remaining:    0.0s
[Parallel(n_jobs

('2', '4', '5', '6', '7')

#SFFS+SMOTE+ADABOOST

In [11]:
feat_cols = list(sfs1.k_feature_idx_)

print(feat_cols)
abc2= AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=4,min_samples_leaf=1, min_samples_split=2 ), learning_rate=1, n_estimators=1000)
# Train Adaboost Classifer
abc2.fit(X_train_res[:,feat_cols], y_train_res)
y_predadb = abc2.predict(X_test[:,feat_cols])
# accuracy_score(y_test,y_predadb)
p=precision_score(y_test, y_predadb, average='micro')
r=recall_score(y_test, y_predadb,average='micro')
f=f1_score(y_test, y_predadb,average='micro')

sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, y_predadb)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))



[2, 4, 5, 6, 7]
Accuracy:    95.52%
Sensitivity:   95.52%
Specificity:  98.33%
Precision:  95.52%
Recall:  95.52%
F1_score: 95.52%
AUC: 96.93%


#MLP Classifier

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

clf= MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=500, activation = 'relu',solver='adam',random_state=14,validation_fraction=0.2)
sfs0 = SFS(clf,  
           k_features=5, 
           forward=True, # if forward = True then SFS otherwise SBS
           floating=True, 
           verbose=2,
           scoring='accuracy', cv=0)
           

#after applying sfs fit the data:
sfs0.fit(X_train, y_train)
sfs0.k_feature_names_  


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   22.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  2.1min finished

[2021-02-21 17:25:44] Features: 1/5 -- score: 0.9406725979672339[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   25.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:  1.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   21.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   21.4s finished

[2021-02-21 17:27:45] Features: 2/5 -- score: 0.9492240361636578[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.2s remaining:    0.0s
[Parallel(n_jobs

('1', '2', '3', '4', '7')

In [13]:
feat_cols = list(sfs0.k_feature_idx_)
print(feat_cols)
classifier= MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=500,activation = 'relu',alpha=1e-5,solver='adam',random_state=16)
classifier.fit(X_train_res[:,feat_cols], y_train_res)
y_predmlp1 = classifier.predict(X_test[:,feat_cols])
accuracy_score(y_test,y_predmlp1)
p=precision_score(y_test, y_predmlp1 , average='micro')
r=recall_score(y_test, y_predmlp1 , average='micro')
f=f1_score(y_test, y_predmlp1 , average='micro')

sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, y_predmlp1)

print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))



[1, 2, 3, 4, 7]
Accuracy:    92.79%
Sensitivity:   92.79%
Specificity:  93.79%
Precision:  92.79%
Recall:  92.79%
F1_score: 92.79%
AUC: 93.29%


#XGBoost Classifier

In [14]:
from xgboost import XGBClassifier
clf = XGBClassifier(learning_rate=0.01,n_estimators=100, random_state=256)
sfs2 = SFS(clf,  
           k_features=5, 
           forward=True, # if forward = True then SFS otherwise SBS
           floating=True, 
           verbose=2,
           scoring='accuracy',cv=0)
           

#after applying sfs fit the data:
sfs2.fit(X_train, y_train)
sfs2.k_feature_names_  


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    5.9s finished

[2021-02-21 17:34:52] Features: 1/5 -- score: 0.9406725979672339[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    6.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s finished

[2021-02-21 17:34:59] Features: 2/5 -- score: 0.9492240361636578[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs

('2', '3', '4', '5', '7')

In [15]:
feat_cols = list(sfs2.k_feature_idx_)
from xgboost import XGBClassifier
print(feat_cols)
xgboost1 = XGBClassifier(max_depth=7,learning_rate=0.1,n_estimators=1000,random_state=178, subsample= 0.9,colsample_bytree=1,
gamma= 5)
xgboost1.fit(X_train_res[:,feat_cols], y_train_res)
# make predictions for test data
predxgboost1 = xgboost1.predict(X_test[:,feat_cols])
accuracy_score(y_test,predxgboost1)
p=precision_score(y_test, predxgboost1,average='micro')
r=recall_score(y_test, predxgboost1,average='micro')
f=f1_score(y_test, predxgboost1,average='micro')
sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, predxgboost1)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))

[2, 3, 4, 5, 7]
Accuracy:    95.81%
Sensitivity:   95.81%
Specificity:  98.29%
Precision:  95.81%
Recall:  95.81%
F1_score: 95.81%
AUC: 97.05%


#SVM

In [16]:
from sklearn import svm
clf = svm.SVC(kernel='linear') # Linear Kernel
sfs3 = SFS(clf,  
           k_features=5, 
           forward=True, # if forward = True then SFS otherwise SBS
           floating=True, 
           verbose=2,
           scoring='accuracy',cv=0)
           

#after applying sfs fit the data:
sfs3.fit(X_train, y_train)
sfs3.k_feature_names_

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.3min finished

[2021-02-21 17:37:36] Features: 1/5 -- score: 0.9406725979672339[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:   55.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.6s finished

[2021-02-21 17:38:40] Features: 2/5 -- score: 0.9492240361636578[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.3s remaining:    0.0s
[Parallel(n_jobs

('0', '2', '3', '4', '7')

In [17]:
feat_cols = list(sfs3.k_feature_idx_)
print(feat_cols)

SVM1 = svm.SVC(kernel='linear',C=0.1) # Linear Kernel
SVM1.fit(X_train_res[:, feat_cols], y_train_res)

#Predict the response for test dataset
y_predsvm1 = SVM1.predict(X_test[:,feat_cols])
p=precision_score(y_test, y_predsvm1,average='micro')
r=recall_score(y_test, y_predsvm1,average='micro')
f=f1_score(y_test, y_predsvm1,average='micro')
sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, y_predsvm1)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))

[0, 2, 3, 4, 7]
Accuracy:    87.00%
Sensitivity:   87.00%
Specificity:  87.41%
Precision:  87.00%
Recall:  87.00%
F1_score: 87.00%
AUC: 87.20%


#Random Forest Classifier



In [18]:
clf=RandomForestClassifier(n_estimators=1000,max_depth=4)

sfs4 = SFS(clf,  
           k_features=7, 
           forward=True, # if forward = True then SFS otherwise SBS
           floating=True, 
           verbose=2,
           scoring='accuracy',cv=0)
           

#after applying sfs fit the data:
sfs4.fit(X_train, y_train)
sfs4.k_feature_names_  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.4min finished

[2021-02-21 17:44:43] Features: 1/7 -- score: 0.9406725979672339[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:  1.3min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.4s finished

[2021-02-21 17:46:13] Features: 2/7 -- score: 0.9492240361636578[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.0s remaining:    0.0s
[Parallel(n_jobs

('0', '1', '2', '3', '4', '5', '7')

In [19]:
feat_cols = list(sfs4.k_feature_idx_)
print(feat_cols)

[0, 1, 2, 3, 4, 5, 7]


In [20]:

#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier
RFC1=RandomForestClassifier(n_estimators=500,max_depth=4,min_samples_leaf=2)
RFC1.fit(X_train_res[:,feat_cols],y_train_res)
y_predrf1=RFC1.predict(X_test[:,feat_cols])
p=precision_score(y_test, y_predrf1 , average='micro')
r=recall_score(y_test, y_predrf1 , average='micro')
f=f1_score(y_test, y_predrf1, average='micro')
sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, y_predrf1)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))


Accuracy:    91.96%
Sensitivity:   91.96%
Specificity:  92.83%
Precision:  91.96%
Recall:  91.96%
F1_score: 91.96%
AUC: 92.40%


#Logistic Regression

In [21]:
clf= LogisticRegression()

sfs5 = SFS(clf,  
           k_features=5, 
           forward=True, # if forward = True then SFS otherwise SBS
           floating=True, 
           verbose=2,
           scoring='accuracy',cv=0)
           

#after applying sfs fit the data:
sfs5.fit(X_train, y_train)
sfs5.k_feature_names_  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.9s finished

[2021-02-21 17:55:00] Features: 1/5 -- score: 0.9406725979672339[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished

[2021-02-21 17:55:01] Features: 2/5 -- score: 0.9492240361636578[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs

('0', '2', '3', '4', '7')

In [22]:
feat_cols = list(sfs5.k_feature_idx_)
print(feat_cols)
lr1 = LogisticRegression(random_state=999)
lr1.fit(X_train_res[:,feat_cols],y_train_res)
y_predlr1=lr1.predict(X_test[:,feat_cols])
p=precision_score(y_test, y_predlr1 , average='micro')
r=recall_score(y_test, y_predlr1,average='micro')
f=f1_score(y_test, y_predlr1, average='micro')

sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, y_predlr1)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))

[0, 2, 3, 4, 7]
Accuracy:    87.00%
Sensitivity:   87.00%
Specificity:  87.41%
Precision:  87.00%
Recall:  87.00%
F1_score: 87.00%
AUC: 87.20%



#Decision Tree classifier

In [23]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
clf = DecisionTreeClassifier()
sfs7 = SFS(clf,  
           k_features=5, 
           forward=True, # if forward = True then SFS otherwise SBS
           floating=True, 
           verbose=2, 
           scoring='accuracy',cv=0)
           

#after applying sfs fit the data:
sfs7.fit(X_train, y_train)
sfs7.k_feature_names_  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished

[2021-02-21 17:55:06] Features: 1/5 -- score: 0.9406725979672339[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished

[2021-02-21 17:55:06] Features: 2/5 -- score: 0.9492240361636578[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs

('0', '2', '3', '4', '7')

In [24]:
feat_cols = list(sfs7.k_feature_idx_)
print(feat_cols)
dtc1 = DecisionTreeClassifier(max_depth=3)
dtc1.fit(X_train_res[:,feat_cols],y_train_res)
pred_dtc1=dtc1.predict(X_test[:,feat_cols])
p=precision_score(y_test, pred_dtc1 , average='micro')
r=recall_score(y_test, pred_dtc1 ,average='micro')
f=f1_score(y_test, pred_dtc1 , average='micro')

sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, pred_dtc1)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))

[0, 2, 3, 4, 7]
Accuracy:    86.79%
Sensitivity:   86.79%
Specificity:  87.45%
Precision:  86.79%
Recall:  86.79%
F1_score: 86.79%
AUC: 87.12%


# Gradient Boosting

In [25]:

clf = GradientBoostingClassifier(learning_rate=0.001,n_estimators=50,random_state=6)

sfs8 = SFS(clf,  
           k_features=5, 
           forward=True, # if forward = True then SFS otherwise SBS
           floating=True, 
           verbose=2,
           scoring='accuracy',cv=0)
           

#after applying sfs fit the data:
sfs8.fit(X_train, y_train)
sfs8.k_feature_names_  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    3.6s finished

[2021-02-21 17:55:10] Features: 1/5 -- score: 0.9143128617085601[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    3.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s finished

[2021-02-21 17:55:14] Features: 2/5 -- score: 0.9143128617085601[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs

('0', '1', '2', '3', '4')

In [26]:
feat_cols = list(sfs8.k_feature_idx_)
print(feat_cols)

[0, 1, 2, 3, 4]


In [27]:
GBC11=GradientBoostingClassifier(learning_rate=0.1,n_estimators=100,max_depth=3)
GBC11.fit(X_train_res[:,feat_cols], y_train_res)
y_test_pred11 = GBC11.predict(X_test[:,feat_cols])
#accuracy_score(y_test,y_test_pred)
p=precision_score(y_test, y_test_pred11 , average='micro')
r=recall_score(y_test, y_test_pred11 , average='micro')
f=f1_score(y_test, y_test_pred11, average='micro')
sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, y_test_pred11)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))


Accuracy:    85.82%
Sensitivity:   85.82%
Specificity:  86.96%
Precision:  85.82%
Recall:  85.82%
F1_score: 85.82%
AUC: 86.39%
